**CNNによる2次元顔画像表情判定**

In [2]:
!ps aux
!kill -9 <pid>

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.2  0.0    992     4 ?        Ss   22:37   0:00 /sbin/docker-
root           7  0.2  0.3 337764 48264 ?        Sl   22:37   0:00 /tools/node/b
root          17  0.2  0.0  35888  4896 ?        Ss   22:37   0:00 tail -n +0 -F
root          39  0.2  0.3 160356 41792 ?        S    22:37   0:00 python3 /usr/
root          52  0.7  0.4 193892 60284 ?        Sl   22:38   0:01 /usr/bin/pyth
root          53  0.0  0.0 706824  7352 ?        Sl   22:38   0:00 /usr/local/bi
root          64  9.2  2.9 2032376 388636 ?      Ssl  22:40   0:03 /usr/bin/pyth
root          84  0.2  0.1 127644 14916 ?        Sl   22:40   0:00 /usr/bin/pyth
root         112  0.0  0.0  59040  6448 ?        R    22:40   0:00 ps aux
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `kill -9 <pid>'


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten,MaxPooling2D,Conv2D
from keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

**訓練画像、検証画像、テスト画像のディレクトリ**

In [5]:
classes = ['angry', 'disgust','fear','happy','neutral','sad','suprise']
nb_classes = len(classes)
#batch_size_for_data_generator = 20

base_dir = '/content/drive/MyDrive/kaggle/Facial Experience'

train_dir = base_dir+'/train'
validation_dir = base_dir+'/valid'
test_dir = base_dir+'/test'

img_rows, img_cols = 48, 48

os.path.join(base_dir,'path')

**ImageDataGeneratorを使って画像データを拡張する**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(img_rows, img_cols),
                                                    color_mode='rgb',
                                                    classes=classes,
                                                    class_mode='categorical',
                                                    batch_size=1117,
                                                    shuffle=True)

train 84892枚
2x2x19x1117
バッジサイズ2048

In [7]:
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)
validation_generator = valid_datagen.flow_from_directory(directory=validation_dir,
                                                        target_size=(img_rows, img_cols),
                                                        color_mode='rgb',
                                                        classes=classes,
                                                        class_mode='categorical',
                                                        batch_size=131,
                                                        shuffle=True)

Found 21615 images belonging to 7 classes.


valid 21615枚 3x5x11x131 バッジサイズ2048

**CNNモデル**

なんで512?→特に理由なし1024でもなんでもいい

dropoutの役割→層から層への伝達の際に出力データを意図的に0とする事で、訓練データに過剰適合する問題を回避する。

出力層
二値分類→sigmoid
他クラス分類→softmax
回帰分類→恒等関数

leaky relu

relu関数においてx<0のとき正の傾きを与える

過学習を起こしたら、入力層にDropout率0.2、隠れ層に0.5を適用する

https://deepage.net/deep_learning/2016/10/17/deeplearning_dropout.html#tflearn%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB

In [8]:
from keras.layers import BatchNormalization
#first layer
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (img_rows, img_cols, 3) ,padding='same'))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
#second layer
model.add(Conv2D(32, (3,3), padding='same' ))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
#regularization
model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
#model.add(Dropout(0.25))

#thirt layer
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
#fourth layer
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
#model.add(Dropout(0.25))

#classification layer
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.25))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

miniVGGnet
https://github.com/matvi/miniVGGNet/blob/master/cnn/MiniVGGNet.py

In [25]:
# model=Sequential()
# # 畳み込み層
# model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(img_rows, img_cols, 3)))
# model.add(MaxPooling2D((2, 2)))

# model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Dropout(0.5))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Dropout(0.5))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Dropout(0.5))

# # 全結合層
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))

# model.add(Dropout(0.2))

# # 出力層
# model.add(Dense(nb_classes, activation='softmax'))
          
# model.summary()

モデル参照先
https://sciresol.s3.us-east-2.amazonaws.com/IJST/Articles/2019/Issue-24/Article9.pdf

In [9]:
opt = Adam(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer= opt, metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


**コールバックを使った学習**

In [10]:
hdf5_file = os.path.join(base_dir, 'model.hdf5')

再度ここから

In [1]:
from keras.callbacks import ModelCheckpoint
modelCheckpoint = ModelCheckpoint(filepath = hdf5_file,
                                  monitor='loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=False,
                                  mode='min',
                                  period=1)

NameError: ignored

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch=76,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=165,
                    callbacks=[modelCheckpoint],
                    verbose=1)

In [ ]:
model.save(hdf5_file)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

**テスト**

In [ ]:
#test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
#model=keras.models.load_model(hdf5_file)

In [ ]:
# import numpy as np
# test_generator = test_datagen.flow_from_directory(directory=test_dir,
#                                                   target_size=(img_rows, img_cols),
#                                                   color_mode='rgb',
#                                                   classes=classes,
#                                                   class_mode='categorical',
#                                                   batch_size=32,
#                                                   shuffle=False)

# test_steps_per_Epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)

In [ ]:
# predictions = model.predict_generator(test_generator, steps=test_steps_per_Epoch)
# # Get most likely class
# predicted_classes = np.argmax(predictions, axis=1)
# # Ground-Truthクラスとクラスラベルを取得する
# true_classes = test_generator.classes
# class_labels = list(test_generator.class_indices.keys())

適合率（precision）: precision_score()：実際に正/正と予測

再現率（recall）: recall_score()：正と予測/実際に正

F1値（F1-measure）: f1_score()：0-1

In [ ]:
# from sklearn import metrics
# # scikit-learnを使用して統計を取得する
# report = metrics.classification_report(true_classes, 
#                                        predicted_classes, 
#                                        labels=np.arange(len(classes)),
#                                        target_names=classes)
# print(report)

**参考**


*   [Keras / CNN] 多クラス画像分類 --- ラーメンの味分類

    https://qiita.com/Phoeboooo/items/cfe8560fe8a285855340
